# Import library

In [1]:
import os
import os, sys
import glob
import re
import datetime
from datetime import datetime, timedelta 
import math, copy, time
from itertools import cycle
from collections import defaultdict

from parse import parse
import numpy as np
import pandas as pd

import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

# import torchvision
# import torchvision.transforms as transforms

from sklearn.metrics import roc_auc_score, plot_roc_curve
from sklearn import svm


from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn

%load_ext autoreload
%autoreload 2

/Users/mckim/miniconda3/envs/py38/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
pd.set_option("display.max_column", 100)

In [3]:
# Util functions
def size_and_ratio(df, col, dropna=True):
    if pd.__version__ > '1.1':
        sr = df.groupby(col, dropna=dropna).size().sort_values(ascending=False)
    elif dropna:
        sr = df.groupby(col).size().sort_values(ascending=False)
    else:  # dropna == False
        sr = df.fillna('nan').groupby(col).size().sort_values(ascending=False)
    sr_ratio = sr.copy() / sum(sr)
    print("Sum :", sum(sr), sr.shape)
    sr.name = 'size'
    sr_ratio.name = 'ratio'
    return pd.concat([sr, sr_ratio], axis=1)

# Data load

In [4]:
df_train = pd.read_csv("data/df_data_v3_1/df_train_v3_1.csv")
df_val = pd.read_csv("data/df_data_v3_1/df_valid_v3_1.csv")
df_test = pd.read_csv("data/df_data_v3_1/df_test_v3_1.csv")
print(df_train.shape, df_val.shape, df_test.shape)
df_train.head()

(14093, 24) (4647, 24) (4586, 24)


,baby_id,cough,fever,sore_throat,shortness_of_breath,head_ache,runny_nose,muscle_pain,chills,loss_of_taste,loss_of_smell,sputum,chest_pain,indication_other,indication_abroad,indication_contact,global_confirmed_ratio,confirmed_ratio,sigungu_confirmed_ratio,mask,gender,age_ratio,weekday,pcr_result
0,21099,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0.173311,0.007956,0.012987,1,0.0,0.18,3,0
1,21151,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0.179009,0.008614,0.013084,1,0.0,0.17,4,0
2,21024,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0.179009,0.008614,0.013084,1,1.0,0.29,4,0
3,21176,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0.131500,0.008252,0.000000,0,0.0,0.40,5,0
4,4467,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0.131500,0.008252,0.000000,0,1.0,0.34,5,0


### Add CLS token

In [5]:
df_train['[CLS]'] = 0
df_val['[CLS]'] = 0
df_test['[CLS]'] = 0

In [6]:
import random
print("Train:", df_train.shape, df_train['baby_id'].nunique())
print("Validaiton:", df_val.shape, df_val['baby_id'].nunique())

Train: (14093, 25) 9641
Validaiton: (4647, 25) 3213


## Remove baby_id and results

In [7]:
exclude_cols = ['baby_id', 'pcr_result']
cols = [x for x in df_train.columns if x not in exclude_cols]

# move [CLS] at first
cols = ['[CLS]'] + [x for x in cols if x !='[CLS]']

df_train_gt = df_train[['pcr_result']]
df_train_input = df_train[cols]
df_val_gt = df_val[['pcr_result']]
df_val_input = df_val[cols]
df_test_gt = df_test[['pcr_result']]
df_test_input = df_test[cols]


In [8]:
cols

['[CLS]',
 'cough',
 'fever',
 'sore_throat',
 'shortness_of_breath',
 'head_ache',
 'runny_nose',
 'muscle_pain',
 'chills',
 'loss_of_taste',
 'loss_of_smell',
 'sputum',
 'chest_pain',
 'indication_other',
 'indication_abroad',
 'indication_contact',
 'global_confirmed_ratio',
 'confirmed_ratio',
 'sigungu_confirmed_ratio',
 'mask',
 'gender',
 'age_ratio',
 'weekday']

# Make category values

In [9]:
from model.attention_model import CategoricalAttentionModel
from model.utils import get_dict_category_from_dataset
from model.utils import get_category_key, create_cat_and_intensity_from_df

In [10]:
df_trainval = pd.concat([df_train, df_val], axis=0)
print(df_trainval.shape)

dict_category, df_trainval_typed = get_dict_category_from_dataset(df_trainval[cols])
_, df_train_typed = get_dict_category_from_dataset(df_train[cols])
_, df_val_typed = get_dict_category_from_dataset(df_val[cols])
_, df_test_typed = get_dict_category_from_dataset(df_test[cols])
dict_category

(18740, 25)
[CLS] category [0] ...
cough category [1 0] ...
fever category [0 1] ...
sore_throat category [0 1] ...
shortness_of_breath category [0 1] ...
head_ache category [1 0] ...
runny_nose category [0 1] ...
muscle_pain category [0 1] ...
chills category [0 1] ...
loss_of_taste category [0 1] ...
loss_of_smell category [0 1] ...
sputum category [1 0] ...
chest_pain category [0 1] ...
indication_other category [0 1] ...
indication_abroad category [0 1] ...
indication_contact category [1 0] ...
global_confirmed_ratio float [0.1733114  0.17900909 0.13149977 0.14631498 0.15655098 0.17425644
 0.17103502 0.16990724 0.12749546 0.14885249]
confirmed_ratio float [0.00795568 0.00861396 0.00825183 0.00797017 0.01154644 0.01142734
 0.0113018  0.01122937 0.01076423 0.00895517]
sigungu_confirmed_ratio float [0.01298674 0.01308374 0.         0.01324623 0.01342084 0.0135712
 0.01798011 0.01397984 0.01779546 0.0141302 ]
mask category [1 0] ...
gender category [0. 1.] ...
age_ratio float [0.18 0.1

{'[CLS]_0.0': 0,
 'cough_1.0': 1,
 'cough_0.0': 2,
 'fever_0.0': 3,
 'fever_1.0': 4,
 'sore_throat_0.0': 5,
 'sore_throat_1.0': 6,
 'shortness_of_breath_0.0': 7,
 'shortness_of_breath_1.0': 8,
 'head_ache_1.0': 9,
 'head_ache_0.0': 10,
 'runny_nose_0.0': 11,
 'runny_nose_1.0': 12,
 'muscle_pain_0.0': 13,
 'muscle_pain_1.0': 14,
 'chills_0.0': 15,
 'chills_1.0': 16,
 'loss_of_taste_0.0': 17,
 'loss_of_taste_1.0': 18,
 'loss_of_smell_0.0': 19,
 'loss_of_smell_1.0': 20,
 'sputum_1.0': 21,
 'sputum_0.0': 22,
 'chest_pain_0.0': 23,
 'chest_pain_1.0': 24,
 'indication_other_0.0': 25,
 'indication_other_1.0': 26,
 'indication_abroad_0.0': 27,
 'indication_abroad_1.0': 28,
 'indication_contact_1.0': 29,
 'indication_contact_0.0': 30,
 'global_confirmed_ratio': 31,
 'confirmed_ratio': 32,
 'sigungu_confirmed_ratio': 33,
 'mask_1.0': 34,
 'mask_0.0': 35,
 'gender_0.0': 36,
 'gender_1.0': 37,
 'age_ratio': 38,
 'weekday_3.0': 39,
 'weekday_4.0': 40,
 'weekday_5.0': 41,
 'weekday_0.0': 42,
 'weekd

### category 값, intensity 값 생성

### train 데이터 먼저

In [11]:
df_train_category, df_train_intensity = create_cat_and_intensity_from_df(df_train_typed, dict_category)

display(df_train_category.sample(10))
display(df_train_intensity.sample(10))

,[CLS],cough,fever,sore_throat,shortness_of_breath,head_ache,runny_nose,muscle_pain,chills,loss_of_taste,loss_of_smell,sputum,chest_pain,indication_other,indication_abroad,indication_contact,global_confirmed_ratio,confirmed_ratio,sigungu_confirmed_ratio,mask,gender,age_ratio,weekday
6523,0.0,1.0,4.0,6.0,7.0,9.0,11.0,13.0,15.0,17.0,19.0,21.0,23.0,26.0,27.0,30.0,31.0,32.0,33.0,34.0,37.0,38.0,40.0
761,0.0,1.0,3.0,6.0,7.0,9.0,11.0,13.0,15.0,17.0,19.0,22.0,23.0,26.0,27.0,30.0,31.0,32.0,33.0,34.0,36.0,38.0,43.0
13827,0.0,2.0,3.0,5.0,7.0,10.0,11.0,13.0,15.0,17.0,19.0,22.0,23.0,26.0,27.0,30.0,31.0,32.0,33.0,34.0,37.0,38.0,44.0
896,0.0,2.0,3.0,5.0,7.0,10.0,11.0,13.0,15.0,17.0,19.0,22.0,23.0,26.0,27.0,30.0,31.0,32.0,33.0,34.0,37.0,38.0,44.0
9422,0.0,1.0,3.0,6.0,7.0,9.0,12.0,13.0,15.0,17.0,19.0,21.0,23.0,26.0,27.0,30.0,31.0,32.0,33.0,34.0,36.0,38.0,41.0
12549,0.0,2.0,3.0,5.0,7.0,10.0,11.0,13.0,15.0,17.0,19.0,22.0,23.0,26.0,27.0,30.0,31.0,32.0,33.0,34.0,37.0,38.0,43.0
2130,0.0,1.0,3.0,6.0,7.0,9.0,11.0,13.0,15.0,17.0,19.0,22.0,23.0,26.0,27.0,30.0,31.0,32.0,33.0,34.0,36.0,38.0,44.0
4373,0.0,1.0,3.0,6.0,7.0,9.0,11.0,13.0,15.0,17.0,19.0,21.0,23.0,26.0,27.0,30.0,31.0,32.0,33.0,34.0,36.0,38.0,44.0
13640,0.0,1.0,3.0,5.0,7.0,10.0,11.0,13.0,15.0,17.0,19.0,22.0,23.0,26.0,27.0,30.0,31.0,32.0,33.0,34.0,37.0,38.0,43.0
10983,0.0,1.0,3.0,5.0,7.0,10.0,11.0,13.0,15.0,17.0,19.0,21.0,23.0,26.0,27.0,30.0,31.0,32.0,33.0,34.0,37.0,38.0,43.0


,[CLS],cough,fever,sore_throat,shortness_of_breath,head_ache,runny_nose,muscle_pain,chills,loss_of_taste,loss_of_smell,sputum,chest_pain,indication_other,indication_abroad,indication_contact,global_confirmed_ratio,confirmed_ratio,sigungu_confirmed_ratio,mask,gender,age_ratio,weekday
5370,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.267451,0.336654,0.242308,1.0,1.0,0.35,1.0
13231,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.164664,0.144094,0.000000,1.0,1.0,0.39,1.0
9597,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.145017,0.146347,0.334325,1.0,1.0,0.26,1.0
1435,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.362753,0.392090,0.082167,1.0,1.0,0.34,1.0
8144,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.261600,0.377104,0.000000,1.0,1.0,0.53,1.0
7687,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.390961,0.516121,0.000000,1.0,1.0,0.39,1.0
12428,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.279611,0.173581,0.469518,1.0,1.0,0.27,1.0
8247,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.261600,0.377104,0.000000,1.0,1.0,0.27,1.0
2094,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.426039,0.551131,0.000000,1.0,1.0,0.44,1.0
1903,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.426039,0.551131,0.108614,1.0,1.0,0.38,1.0


예시로 sore_throat id값 확인해봄

In [12]:
size_and_ratio(df_train_category, 'sore_throat')

Sum : 14093 (2,)


,size,ratio
sore_throat,,
6.0,7723,0.548003
5.0,6370,0.451997


### df_val에서 df_cat값과 df_intensity 값 생성

In [13]:
df_val_category, df_val_intensity = create_cat_and_intensity_from_df(df_val_typed, dict_category)


### df_test에서도 마찬가지로 생성

In [14]:
df_test_category, df_test_intensity = create_cat_and_intensity_from_df(df_test_typed, dict_category)


In [15]:
train_np_cat = df_train_category.values
train_np_ins = df_train_intensity.values
train_np_gt = df_train_gt[['pcr_result']].values

val_np_cat = df_val_category.values
val_np_ins = df_val_intensity.values
val_np_gt = df_val_gt[['pcr_result']].values

test_np_cat = df_test_category.values
test_np_ins = df_test_intensity.values
test_np_gt = df_test_gt[['pcr_result']].values

train_np_gt, val_np_gt, test_np_gt

(array([[0],
        [0],
        [0],
        ...,
        [1],
        [0],
        [0]]),
 array([[0],
        [0],
        [0],
        ...,
        [0],
        [1],
        [0]]),
 array([[0],
        [0],
        [0],
        ...,
        [1],
        [0],
        [1]]))

In [16]:
train_input = [train_np_cat.copy(), train_np_ins.copy()]
train_gt = train_np_gt.copy()

valid_input = [val_np_cat.copy(), val_np_ins.copy()]
valid_gt = val_np_gt.copy()

test_input = [test_np_cat.copy(), test_np_ins.copy()]
test_gt = test_np_gt.copy()


In [17]:
print(train_input[0].shape, valid_input[0].shape, test_input[0].shape)
print(train_gt.shape, valid_gt.shape, test_gt.shape)

(14093, 23) (4647, 23) (4586, 23)
(14093, 1) (4647, 1) (4586, 1)


In [18]:
print("# of train data positive: ", train_gt.sum())
print("# of train data negative: ", train_gt.shape[0] - train_gt.sum())
print("ratio of P/N: ", train_gt.sum()/(train_gt.shape[0] - train_gt.sum()))
print("")

print("# of valid data positive: ", valid_gt.sum())
print("# of valid data negative: ", valid_gt.shape[0] - valid_gt.sum())
print("ratio of P/N: ", valid_gt.sum()/(valid_gt.shape[0] - valid_gt.sum()))
print("")

print("# of test data positive: ", test_gt.sum())
print("# of test data negative: ", test_gt.shape[0] - test_gt.sum())
print("ratio of P/N: ", test_gt.sum()/(test_gt.shape[0] - test_gt.sum()))

# of train data positive:  10063
# of train data negative:  4030
ratio of P/N:  2.4970223325062033

# of valid data positive:  3415
# of valid data negative:  1232
ratio of P/N:  2.7719155844155843

# of test data positive:  3332
# of test data negative:  1254
ratio of P/N:  2.657097288676236


### Check dict_category

In [19]:
dict_category

{'[CLS]_0.0': 0,
 'cough_1.0': 1,
 'cough_0.0': 2,
 'fever_0.0': 3,
 'fever_1.0': 4,
 'sore_throat_0.0': 5,
 'sore_throat_1.0': 6,
 'shortness_of_breath_0.0': 7,
 'shortness_of_breath_1.0': 8,
 'head_ache_1.0': 9,
 'head_ache_0.0': 10,
 'runny_nose_0.0': 11,
 'runny_nose_1.0': 12,
 'muscle_pain_0.0': 13,
 'muscle_pain_1.0': 14,
 'chills_0.0': 15,
 'chills_1.0': 16,
 'loss_of_taste_0.0': 17,
 'loss_of_taste_1.0': 18,
 'loss_of_smell_0.0': 19,
 'loss_of_smell_1.0': 20,
 'sputum_1.0': 21,
 'sputum_0.0': 22,
 'chest_pain_0.0': 23,
 'chest_pain_1.0': 24,
 'indication_other_0.0': 25,
 'indication_other_1.0': 26,
 'indication_abroad_0.0': 27,
 'indication_abroad_1.0': 28,
 'indication_contact_1.0': 29,
 'indication_contact_0.0': 30,
 'global_confirmed_ratio': 31,
 'confirmed_ratio': 32,
 'sigungu_confirmed_ratio': 33,
 'mask_1.0': 34,
 'mask_0.0': 35,
 'gender_0.0': 36,
 'gender_1.0': 37,
 'age_ratio': 38,
 'weekday_3.0': 39,
 'weekday_4.0': 40,
 'weekday_5.0': 41,
 'weekday_0.0': 42,
 'weekd

## Import model

In [25]:
from model.base_dataloader import IsraelDataLoader, IsraelDatasetWithIntensity
from model.utils import run_validation

In [26]:
print("cat value:", train_input[0].shape)
print("intensity value:", train_input[1].shape)
print("Ground truth value:", train_gt.shape)

cat value: (14093, 23)
intensity value: (14093, 23)
Ground truth value: (14093, 1)


# Dataset, data_loader

In [27]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [28]:
# creat dataset

train_dataset = IsraelDatasetWithIntensity(
    train_np_cat,
    train_np_ins,
    train_np_gt,
    dict_category
)

val_dataset = IsraelDatasetWithIntensity(
    val_np_cat,
    val_np_ins,
    val_np_gt,
    dict_category
)

test_dataset = IsraelDatasetWithIntensity(
    test_np_cat, 
    test_np_ins,
    test_np_gt,
    dict_category
)

## Transformer encoder
3층

In [41]:
from model import transformer_encoder_model

In [42]:
def save_model(config_name, model, epoch):
    save_path = f"./train_data/{model.__class__.__name__}_{config_name}_{epoch:04d}.pth"
    print("save_path", save_path)
    torch.save(model.state_dict(), save_path)
    print("Success to save to : ", save_path)
    return model

batch_size = 1024
epochs = 300

train_loader = IsraelDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = IsraelDataLoader(val_dataset, batch_size=batch_size, shuffle=False)

len_seq = train_input[0].shape[1]
print("len_seq:", len_seq)
d_model = 32
n_head = 4
n_layer = 2
lr = 1e-4

encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_head, dropout=0.1)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layer)
model = transformer_encoder_model.MuliplyIntensityUsingCls(
    transformer_encoder=transformer_encoder,
    dict_category=dict_category,
    len_seq=len_seq,
)

model.eval()

print("d_model:", model.d_model, "n_head:", model.n_head, 'len_seq:', model.len_seq)
loss_func = nn.BCELoss(reduction='mean')
optimizer = optim.RAdam(model.parameters(), lr=lr)
config_name = f"big_layer{n_layer}_dmodel{d_model}_nhead{n_head}_lr{lr}"

train_log = dict()
val_log = dict()

val_metrics = {
    "loss": loss_func,
    'auc': lambda pred, gt: roc_auc_score(gt.numpy(), pred.numpy())
}


for epoch in range(epochs):
    # **************** Train ****************
    train_result_info = transformer_encoder_model.MuliplyIntensityUsingCls.train_on_epoch(epoch, train_loader, model, loss_func, optimizer, verbose=True)
    train_log[f'epoch_train_{epoch}'] = train_result_info
    logger.debug(f"epoch {epoch} train finished. train_result_info:{train_result_info}")
    
    # ************** validation *********************
    val_result = transformer_encoder_model.MuliplyIntensityUsingCls.validation_on_epoch(epoch, model, val_loader, val_metrics)
    val_log[f'epoch_val_{epoch}'] = val_result
    logger.info(f"epoch {epoch} val result: {val_result}")

    if not val_log:
        best_model = save_model(config_name, model.copy(), epoch)
    elif val_result['auc'] >= max(val_log.values(), key=lambda x : x['auc'])['auc']:
        best_model = save_model(config_name, model.copy(), epoch)

    

len_seq: 23
d_model: 32 n_head: 4 len_seq: 23


INFO:model.base_model:Epoch Step: 0 Train Loss: 8.860494613647461 elapsed: 34.00273895263672
INFO:model.base_model:cur_epoch:0, validation elapsed_time:2.054316997528076
INFO:__main__:epoch 0 val result: {'loss': tensor(0.6203), 'auc': 0.42990732729934783}


save_path ./train_data/MuliplyIntensityUsingCls_big_layer2_dmodel32_nhead4_lr2e-05_0000.pth
Success to save to :  ./train_data/MuliplyIntensityUsingCls_big_layer2_dmodel32_nhead4_lr2e-05_0000.pth


INFO:model.base_model:Epoch Step: 1 Train Loss: 8.789953231811523 elapsed: 34.04216694831848
INFO:model.base_model:cur_epoch:1, validation elapsed_time:1.924994945526123
INFO:__main__:epoch 1 val result: {'loss': tensor(0.6127), 'auc': 0.5700926727006521}


save_path ./train_data/MuliplyIntensityUsingCls_big_layer2_dmodel32_nhead4_lr2e-05_0001.pth
Success to save to :  ./train_data/MuliplyIntensityUsingCls_big_layer2_dmodel32_nhead4_lr2e-05_0001.pth


...

INFO:model.base_model:cur_epoch:143, validation elapsed_time:2.4466192722320557
INFO:__main__:epoch 143 val result: {'loss': tensor(0.5811), 'auc': 0.5700926727006521}
INFO:model.base_model:Epoch Step: 144 Train Loss: 8.40902328491211 elapsed: 39.245620012283325
INFO:model.base_model:cur_epoch:144, validation elapsed_time:2.517498016357422
INFO:__main__:epoch 144 val result: {'loss': tensor(0.5813), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 145 Train Loss: 8.395877838134766 elapsed: 40.161771059036255
INFO:model.base_model:cur_epoch:145, validation elapsed_time:2.513010025024414
INFO:__main__:epoch 145 val result: {'loss': tensor(0.5814), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 146 Train Loss: 8.400802612304688 elapsed: 39.75420784950256
INFO:model.base_model:cur_epoch:146, validation elapsed_time:2.422015905380249
INFO:__main__:epoch 146 val result: {'loss': tensor(0.5816), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 147 Train Loss: 8.411212921142578 elapsed: 39.7104120254

INFO:model.base_model:Epoch Step: 176 Train Loss: 8.389928817749023 elapsed: 37.70425319671631
INFO:model.base_model:cur_epoch:176, validation elapsed_time:2.3773679733276367
INFO:__main__:epoch 176 val result: {'loss': tensor(0.5786), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 177 Train Loss: 8.381505012512207 elapsed: 38.71978497505188
INFO:model.base_model:cur_epoch:177, validation elapsed_time:2.2417078018188477
INFO:__main__:epoch 177 val result: {'loss': tensor(0.5786), 'auc': 0.42990732729934783}
INFO:model.base_model:Epoch Step: 178 Train Loss: 8.382375717163086 elapsed: 37.457576274871826
INFO:model.base_model:cur_epoch:178, validation elapsed_time:2.1249477863311768
INFO:__main__:epoch 178 val result: {'loss': tensor(0.5785), 'auc': 0.5004058441558441}
INFO:model.base_model:Epoch Step: 179 Train Loss: 8.388367652893066 elapsed: 38.00968813896179
INFO:model.base_model:cur_epoch:179, validation elapsed_time:2.245134115219116
INFO:__main__:epoch 179 val result: {'loss': tenso

INFO:model.base_model:cur_epoch:208, validation elapsed_time:1.9749419689178467
INFO:__main__:epoch 208 val result: {'loss': tensor(0.5783), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 209 Train Loss: 8.381744384765625 elapsed: 36.55604815483093
INFO:model.base_model:cur_epoch:209, validation elapsed_time:2.2046709060668945
INFO:__main__:epoch 209 val result: {'loss': tensor(0.5783), 'auc': 0.42990732729934783}
INFO:model.base_model:Epoch Step: 210 Train Loss: 8.387661933898926 elapsed: 35.20423126220703
INFO:model.base_model:cur_epoch:210, validation elapsed_time:2.127802848815918
INFO:__main__:epoch 210 val result: {'loss': tensor(0.5783), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 211 Train Loss: 8.38177490234375 elapsed: 37.24608373641968
INFO:model.base_model:cur_epoch:211, validation elapsed_time:2.274869918823242
INFO:__main__:epoch 211 val result: {'loss': tensor(0.5783), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 212 Train Loss: 8.39557933807373 elapsed: 38.28264093399

INFO:model.base_model:Epoch Step: 241 Train Loss: 8.400778770446777 elapsed: 39.46035623550415
INFO:model.base_model:cur_epoch:241, validation elapsed_time:2.618475914001465
INFO:__main__:epoch 241 val result: {'loss': tensor(0.5806), 'auc': 0.5700926727006521}
INFO:model.base_model:Epoch Step: 242 Train Loss: 8.384703636169434 elapsed: 38.41605997085571
INFO:model.base_model:cur_epoch:242, validation elapsed_time:2.386600971221924
INFO:__main__:epoch 242 val result: {'loss': tensor(0.5806), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 243 Train Loss: 8.399829864501953 elapsed: 39.60428071022034
INFO:model.base_model:cur_epoch:243, validation elapsed_time:2.3743209838867188
INFO:__main__:epoch 243 val result: {'loss': tensor(0.5806), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 244 Train Loss: 8.392889022827148 elapsed: 39.73822212219238
INFO:model.base_model:cur_epoch:244, validation elapsed_time:2.2245848178863525
INFO:__main__:epoch 244 val result: {'loss': tensor(0.5806), 'auc': 

INFO:__main__:epoch 273 val result: {'loss': tensor(0.5784), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 274 Train Loss: 8.383217811584473 elapsed: 38.13424587249756
INFO:model.base_model:cur_epoch:274, validation elapsed_time:2.2833001613616943
INFO:__main__:epoch 274 val result: {'loss': tensor(0.5784), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 275 Train Loss: 8.382024765014648 elapsed: 39.267411947250366
INFO:model.base_model:cur_epoch:275, validation elapsed_time:2.166938066482544
INFO:__main__:epoch 275 val result: {'loss': tensor(0.5784), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 276 Train Loss: 8.382658004760742 elapsed: 38.45866274833679
INFO:model.base_model:cur_epoch:276, validation elapsed_time:2.2760987281799316
INFO:__main__:epoch 276 val result: {'loss': tensor(0.5784), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 277 Train Loss: 8.380265235900879 elapsed: 37.931304931640625
INFO:model.base_model:cur_epoch:277, validation elapsed_time:2.178351879119873
INFO:__m

In [43]:
## Check the best model's validation performance
display(val_log)
print("Best validation performance:", max(val_log.values(), key=lambda x : x['auc']))

## Check the best model's test performance
test_loader = IsraelDataLoader(test_dataset, batch_size=64, shuffle=False)

best_model.eval()
test_result = best_model.validation_on_epoch(epoch, best_model, test_loader, val_metrics)
test_result

{'epoch_val_0': {'loss': tensor(0.6203), 'auc': 0.42990732729934783},
 'epoch_val_1': {'loss': tensor(0.6127), 'auc': 0.5700926727006521},
 'epoch_val_2': {'loss': tensor(0.6027), 'auc': 0.5},
 'epoch_val_3': {'loss': tensor(0.5927), 'auc': 0.42990732729934783},
 'epoch_val_4': {'loss': tensor(0.5846), 'auc': 0.49959415584415584},
 'epoch_val_5': {'loss': tensor(0.5797), 'auc': 0.5700926727006521},
 'epoch_val_6': {'loss': tensor(0.5784), 'auc': 0.5},
 'epoch_val_7': {'loss': tensor(0.5804), 'auc': 0.5700926727006521},
 'epoch_val_8': {'loss': tensor(0.5847), 'auc': 0.5},
 'epoch_val_9': {'loss': tensor(0.5900), 'auc': 0.5},
 'epoch_val_10': {'loss': tensor(0.5949), 'auc': 0.5700926727006521},
 'epoch_val_11': {'loss': tensor(0.5987), 'auc': 0.5700926727006521},
 'epoch_val_12': {'loss': tensor(0.6007), 'auc': 0.5700926727006521},
 'epoch_val_13': {'loss': tensor(0.6010), 'auc': 0.5701257106729288},
 'epoch_val_14': {'loss': tensor(0.5995), 'auc': 0.42990732729934783},
 'epoch_val_15':

Best validation performance: {'loss': tensor(0.6010), 'auc': 0.5701257106729288}


INFO:model.base_model:cur_epoch:299, validation elapsed_time:1.1825716495513916


{'loss': tensor(0.6136), 'auc': 0.492582918334798}

In [ ]:
# 무한대로 가보자... 누가이기나!
for epoch in range(epochs,10000):
    # **************** Train ****************
    train_result_info = transformer_encoder_model.MuliplyIntensityUsingCls.train_on_epoch(epoch, train_loader, model, loss_func, optimizer, verbose=True)
    train_log[f'epoch_train_{epoch}'] = train_result_info
    logger.debug(f"epoch {epoch} train finished. train_result_info:{train_result_info}")
    
    # ************** validation *********************
    val_result = transformer_encoder_model.MuliplyIntensityUsingCls.validation_on_epoch(epoch, model, val_loader, val_metrics)
    val_log[f'epoch_val_{epoch}'] = val_result
    logger.info(f"epoch {epoch} val result: {val_result}")

    if not val_log:
        best_model = save_model(config_name, model.copy(), epoch)
    elif val_result['auc'] >= max(val_log.values(), key=lambda x : x['auc'])['auc']:
        best_model = save_model(config_name, model.copy(), epoch)

    

INFO:model.base_model:Epoch Step: 300 Train Loss: 8.390231132507324 elapsed: 39.156742811203
INFO:model.base_model:cur_epoch:300, validation elapsed_time:2.3297760486602783
INFO:__main__:epoch 300 val result: {'loss': tensor(0.5783), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 301 Train Loss: 8.387165069580078 elapsed: 39.05733299255371
INFO:model.base_model:cur_epoch:301, validation elapsed_time:2.17761492729187
INFO:__main__:epoch 301 val result: {'loss': tensor(0.5783), 'auc': 0.430313171455192}
INFO:model.base_model:Epoch Step: 302 Train Loss: 8.380742073059082 elapsed: 38.63583302497864
INFO:model.base_model:cur_epoch:302, validation elapsed_time:2.359701633453369
INFO:__main__:epoch 302 val result: {'loss': tensor(0.5783), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 303 Train Loss: 8.380919456481934 elapsed: 39.15960717201233
INFO:model.base_model:cur_epoch:303, validation elapsed_time:2.7830159664154053
INFO:__main__:epoch 303 val result: {'loss': tensor(0.5783), 'auc': 0.5}

INFO:model.base_model:Epoch Step: 333 Train Loss: 8.39948558807373 elapsed: 38.42145395278931
INFO:model.base_model:cur_epoch:333, validation elapsed_time:2.900789976119995
INFO:__main__:epoch 333 val result: {'loss': tensor(0.5800), 'auc': 0.569686828544808}
INFO:model.base_model:Epoch Step: 334 Train Loss: 8.391724586486816 elapsed: 38.64226198196411
INFO:model.base_model:cur_epoch:334, validation elapsed_time:2.3692946434020996
INFO:__main__:epoch 334 val result: {'loss': tensor(0.5800), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 335 Train Loss: 8.396775245666504 elapsed: 38.50806999206543
INFO:model.base_model:cur_epoch:335, validation elapsed_time:2.305468797683716
INFO:__main__:epoch 335 val result: {'loss': tensor(0.5800), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 336 Train Loss: 8.392192840576172 elapsed: 40.61908197402954
INFO:model.base_model:cur_epoch:336, validation elapsed_time:2.1653358936309814
INFO:__main__:epoch 336 val result: {'loss': tensor(0.5800), 'auc': 0.

INFO:__main__:epoch 365 val result: {'loss': tensor(0.5785), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 366 Train Loss: 8.39291763305664 elapsed: 49.08674716949463
INFO:model.base_model:cur_epoch:366, validation elapsed_time:3.078366994857788
INFO:__main__:epoch 366 val result: {'loss': tensor(0.5784), 'auc': 0.5700926727006521}
INFO:model.base_model:Epoch Step: 367 Train Loss: 8.38166618347168 elapsed: 50.595536947250366
INFO:model.base_model:cur_epoch:367, validation elapsed_time:3.0938448905944824
INFO:__main__:epoch 367 val result: {'loss': tensor(0.5784), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 368 Train Loss: 8.385560989379883 elapsed: 52.048118114471436
INFO:model.base_model:cur_epoch:368, validation elapsed_time:3.1415538787841797
INFO:__main__:epoch 368 val result: {'loss': tensor(0.5784), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 369 Train Loss: 8.387419700622559 elapsed: 50.454286098480225
INFO:model.base_model:cur_epoch:369, validation elapsed_time:3.0733430385

INFO:model.base_model:Epoch Step: 398 Train Loss: 8.388168334960938 elapsed: 38.07135605812073
INFO:model.base_model:cur_epoch:398, validation elapsed_time:2.2035112380981445
INFO:__main__:epoch 398 val result: {'loss': tensor(0.5784), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 399 Train Loss: 8.390657424926758 elapsed: 40.92885088920593
INFO:model.base_model:cur_epoch:399, validation elapsed_time:2.12447190284729
INFO:__main__:epoch 399 val result: {'loss': tensor(0.5784), 'auc': 0.42990732729934783}
INFO:model.base_model:Epoch Step: 400 Train Loss: 8.384673118591309 elapsed: 38.86804413795471
INFO:model.base_model:cur_epoch:400, validation elapsed_time:2.206207036972046
INFO:__main__:epoch 400 val result: {'loss': tensor(0.5785), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 401 Train Loss: 8.388368606567383 elapsed: 39.07753396034241
INFO:model.base_model:cur_epoch:401, validation elapsed_time:3.381493091583252
INFO:__main__:epoch 401 val result: {'loss': tensor(0.5785), 'auc': 0

INFO:__main__:epoch 430 val result: {'loss': tensor(0.5798), 'auc': 0.42990732729934783}
INFO:model.base_model:Epoch Step: 431 Train Loss: 8.383462905883789 elapsed: 36.673035860061646
INFO:model.base_model:cur_epoch:431, validation elapsed_time:2.2440998554229736
INFO:__main__:epoch 431 val result: {'loss': tensor(0.5797), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 432 Train Loss: 8.39499568939209 elapsed: 37.76940202713013
INFO:model.base_model:cur_epoch:432, validation elapsed_time:2.2092931270599365
INFO:__main__:epoch 432 val result: {'loss': tensor(0.5797), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 433 Train Loss: 8.393026351928711 elapsed: 37.69944882392883
INFO:model.base_model:cur_epoch:433, validation elapsed_time:2.5709660053253174
INFO:__main__:epoch 433 val result: {'loss': tensor(0.5796), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 434 Train Loss: 8.3877534866333 elapsed: 37.395421743392944
INFO:model.base_model:cur_epoch:434, validation elapsed_time:2.3768332004

...

INFO:model.base_model:cur_epoch:2542, validation elapsed_time:2.8779571056365967
INFO:__main__:epoch 2542 val result: {'loss': tensor(0.5797), 'auc': 0.5700926727006521}
INFO:model.base_model:Epoch Step: 2543 Train Loss: 8.380044937133789 elapsed: 39.088568925857544
INFO:model.base_model:cur_epoch:2543, validation elapsed_time:2.8075802326202393
INFO:__main__:epoch 2543 val result: {'loss': tensor(0.5797), 'auc': 0.49959415584415584}
INFO:model.base_model:Epoch Step: 2544 Train Loss: 8.386670112609863 elapsed: 39.39053797721863
INFO:model.base_model:cur_epoch:2544, validation elapsed_time:2.757694959640503
INFO:__main__:epoch 2544 val result: {'loss': tensor(0.5796), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 2545 Train Loss: 8.38243293762207 elapsed: 39.802639961242676
INFO:model.base_model:cur_epoch:2545, validation elapsed_time:2.7331910133361816
INFO:__main__:epoch 2545 val result: {'loss': tensor(0.5796), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 2546 Train Loss: 8.38282871

INFO:model.base_model:cur_epoch:2574, validation elapsed_time:2.6271121501922607
INFO:__main__:epoch 2574 val result: {'loss': tensor(0.5785), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 2575 Train Loss: 8.385265350341797 elapsed: 38.54383206367493
INFO:model.base_model:cur_epoch:2575, validation elapsed_time:2.7221601009368896
INFO:__main__:epoch 2575 val result: {'loss': tensor(0.5785), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 2576 Train Loss: 8.384224891662598 elapsed: 35.81435227394104
INFO:model.base_model:cur_epoch:2576, validation elapsed_time:2.261061906814575
INFO:__main__:epoch 2576 val result: {'loss': tensor(0.5785), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 2577 Train Loss: 8.377893447875977 elapsed: 34.59477996826172
INFO:model.base_model:cur_epoch:2577, validation elapsed_time:2.176408052444458
INFO:__main__:epoch 2577 val result: {'loss': tensor(0.5785), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 2578 Train Loss: 8.38471508026123 elapsed: 35.32144904136658

INFO:model.base_model:Epoch Step: 2607 Train Loss: 8.383191108703613 elapsed: 40.58787989616394
INFO:model.base_model:cur_epoch:2607, validation elapsed_time:2.721388101577759
INFO:__main__:epoch 2607 val result: {'loss': tensor(0.5793), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 2608 Train Loss: 8.388727188110352 elapsed: 36.448084115982056
INFO:model.base_model:cur_epoch:2608, validation elapsed_time:2.4879038333892822
INFO:__main__:epoch 2608 val result: {'loss': tensor(0.5793), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 2609 Train Loss: 8.39217758178711 elapsed: 35.70706105232239
INFO:model.base_model:cur_epoch:2609, validation elapsed_time:2.5446999073028564
INFO:__main__:epoch 2609 val result: {'loss': tensor(0.5792), 'auc': 0.5700926727006521}
INFO:model.base_model:Epoch Step: 2610 Train Loss: 8.383784294128418 elapsed: 35.04701471328735
INFO:model.base_model:cur_epoch:2610, validation elapsed_time:2.1293041706085205
INFO:__main__:epoch 2610 val result: {'loss': tensor(0.5

INFO:model.base_model:cur_epoch:2639, validation elapsed_time:2.745126962661743
INFO:__main__:epoch 2639 val result: {'loss': tensor(0.5788), 'auc': 0.5700926727006521}
INFO:model.base_model:Epoch Step: 2640 Train Loss: 8.387500762939453 elapsed: 36.63198709487915
INFO:model.base_model:cur_epoch:2640, validation elapsed_time:2.965406894683838
INFO:__main__:epoch 2640 val result: {'loss': tensor(0.5788), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 2641 Train Loss: 8.376888275146484 elapsed: 36.725627183914185
INFO:model.base_model:cur_epoch:2641, validation elapsed_time:2.498757839202881
INFO:__main__:epoch 2641 val result: {'loss': tensor(0.5789), 'auc': 0.42990732729934783}
INFO:model.base_model:Epoch Step: 2642 Train Loss: 8.382089614868164 elapsed: 38.59776997566223
INFO:model.base_model:cur_epoch:2642, validation elapsed_time:2.8915491104125977
INFO:__main__:epoch 2642 val result: {'loss': tensor(0.5790), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 2643 Train Loss: 8.3815498352

INFO:__main__:epoch 2671 val result: {'loss': tensor(0.5788), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 2672 Train Loss: 8.384239196777344 elapsed: 37.067678928375244
INFO:model.base_model:cur_epoch:2672, validation elapsed_time:2.6974360942840576
INFO:__main__:epoch 2672 val result: {'loss': tensor(0.5787), 'auc': 0.430313171455192}
INFO:model.base_model:Epoch Step: 2673 Train Loss: 8.383577346801758 elapsed: 36.77943706512451
INFO:model.base_model:cur_epoch:2673, validation elapsed_time:2.3846678733825684
INFO:__main__:epoch 2673 val result: {'loss': tensor(0.5787), 'auc': 0.5}
INFO:model.base_model:Epoch Step: 2674 Train Loss: 8.3734130859375 elapsed: 37.57805800437927
INFO:model.base_model:cur_epoch:2674, validation elapsed_time:2.4883999824523926
INFO:__main__:epoch 2674 val result: {'loss': tensor(0.5786), 'auc': 0.49959415584415584}
INFO:model.base_model:Epoch Step: 2675 Train Loss: 8.38420581817627 elapsed: 36.76345920562744
INFO:model.base_model:cur_epoch:2675, validation 

### 6층 짜리로 한번 가보자